In this notebook, you will learn how to make your first submission to the [Tabular Playground Series - Mar 2021 competition.](https://www.kaggle.com/c/tabular-playground-series-mar-2021)

# Make the most of this notebook!

You can use the "Copy and Edit" button in the upper right of the page to create your own copy of this notebook and experiment with different models. You can run it as is and then see if you can make improvements.

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder

from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt
        
input_path = Path('/kaggle/input/tabular-playground-series-mar-2021/')

# Read in the data files

In [ ]:
train = pd.read_csv(input_path / 'train.csv', index_col='id')
display(train.head())

In [ ]:
test = pd.read_csv(input_path / 'test.csv', index_col='id')
display(test.head())

In [ ]:
submission = pd.read_csv(input_path / 'sample_submission.csv', index_col='id')
display(submission.head())

## We need to encode the categoricals.

There are different strategies to accomplish this, and different approaches will have different performance when using different algorithms.  You may decide to encode features with high cardinality (e.g., more distinct values) diffirently than features with low cardinality. For this starter notebook, we'll use simple encoding.

## Pull out the target, and make a validation split

In [ ]:
target = train.pop('target')
X_train_full, X_valid_full, y_train, y_test = train_test_split(train, target, train_size=0.80, test_size=0.20,
                                                   random_state=0)

In [ ]:
categorical_cols_lo_card = [cname for cname in X_train_full.columns if
                    X_train_full[cname].nunique() < 10 and 
                    X_train_full[cname].dtype == "object"]

categorical_cols_hi_card = [cname for cname in X_train_full.columns if
                    X_train_full[cname].nunique() >= 10 and 
                    X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if 
                X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols_lo_card + categorical_cols_hi_card + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

In [ ]:
display(X_train.head())

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer_hi_card = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('lanelencoder', LabelEncoder())
])

# Preprocessing for categorical data
categorical_transformer_lo_card = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat_lo', categorical_transformer_lo_card, categorical_cols_lo_card),
        ('cat_hi', categorical_transformer_lo_card, categorical_cols_hi_card)
    ])

# Define model
model = RandomForestRegressor(n_estimators=100, random_state=0)

# Bundle preprocessing and modeling code in a pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

# Preprocessing of training data, fit model 
clf.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = clf.predict(X_valid)

print('MAE:', mean_absolute_error(y_valid, preds))